In [2]:
# use the raw data :')
import mne
import numpy as np
import pandas as pd
import scipy
import re
import random

# some hyperparameter type things :)
sr = 200
t = 200
ch = 19
affix = '_raw'

In [3]:
# load from .pkl files ("raw")
raw_features = {}
raw_labels = {}
fileList = []

for letter in ['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I']: 
    raw_features[f'subj_{letter}'] = np.load(f'pickles/subj_{letter}_features{affix}.npy')
    raw_labels[f'subj_{letter}'] = np.load(f'pickles/subj_{letter}_labels{affix}.npy').reshape(-1)
    fileList.append((f'subj_{letter}', raw_features[f'subj_{letter}'], raw_labels[f'subj_{letter}']))


In [ ]:
# DONt DO: relabel and resample ?

fileList_thumb = []
fileList_index = []
fileList_middle = []
fileList_ring = []
fileList_pinky = []

newFileList = []

for idx, (subj, features, labels) in enumerate(fileList):
    og_labels = labels
    new_labels = []

    new_labels.append(np.where(labels == 0, 1, 0)) # thumb
    new_labels.append(np.where(labels == 1, 1, 0)) # index
    new_labels.append(np.where(labels == 2, 1, 0)) # middle
    new_labels.append(np.where(labels == 3, 1, 0)) # ring
    new_labels.append(np.where(labels == 4, 1, 0)) # pinky

    fileList[idx] = (subj, features, new_labels)
    
    # print('resampled dataset shape', Counter(y_res))
    

In [4]:
# ConvNet_Finger

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

class ConvNet_Finger(nn.Module):
    def __init__(self):
        super(ConvNet_Finger, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, (7, 3)),
            nn.ReLU(), 
            #nn.MaxPool2d(2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, (5, 3)),
            nn.Conv2d(16, 32, (3, 3)),
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.conv3 = nn.Sequential(
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(43456, 256),
            nn.ReLU(),
            nn.Dropout(0.4), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 2),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        ### FILL IN ### [5 POINTS]
        x = self.conv1(x)
        x = self.conv2(x)
        # x = self.conv3(x)
        x = self.hidden(x)
        return x


In [4]:
# ConvNet_2D

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

class ConvNet_2D(nn.Module):
    def __init__(self):
        super(ConvNet_2D, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, (7, 3)),
            nn.ReLU(), 
            #nn.MaxPool2d(2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, (5, 3)),
            nn.Conv2d(16, 32, (3, 3)),
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.conv3 = nn.Sequential(
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(43456, 256),
            nn.ReLU(),
            nn.Dropout(0.4), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 5),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        ### FILL IN ### [5 POINTS]
        x = self.conv1(x)
        x = self.conv2(x)
        # x = self.conv3(x)
        x = self.hidden(x)
        return x


In [5]:
# final NeuralNet
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(43456, 256),
            nn.ReLU(),
            nn.Dropout(0.4), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 5),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        nn.Sequential(
            nn.Linear(5, 16),
            nn.Linear(16, 32),
            nn.Linear(32, 5)
        )
        return x


In [6]:
# helper functions

def train(train_loader, n_epochs):
    for epoch in range(n_epochs):
        for batch_idx, (data, targets) in enumerate(train_loader):
            # print('batch = ', batch_idx)
            data = data.to(device)
            targets = targets.to(device)

            model.train()
            predictions = model(data)
            loss = criterion(predictions, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    return model


def predict(y_train):
    data = y_train
    pred = []
    with torch.no_grad():
        model.eval()
        data = data.to(device)
        predictions = model(data)
        _, predictions = predictions.max(1)
    return predictions

def evaluate(predictions, targets):
    n_samples, n_correct = 0, 0
    # print(targets.shape, predictions.shape)
    # print(type(targets), targets.shape, type(predictions), len(predictions))
    n_samples += targets.size(0)
    n_correct += (predictions == targets).sum().item()
    acc = n_correct / n_samples
    print(f'accuracy = {acc}')
    return acc

from torch.utils.data import Dataset, DataLoader

class EEGDataset(Dataset):
    def __init__(self, features, labels):
        super(Dataset, self).__init__()
        self.features = features
        self.labels = labels
        self.shape = features.shape, labels.shape
    
    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [ ]:
# i tried to change the code but i dont think this is correct :D
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
# train neural net
torch.cuda.set_device(3)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sm = SMOTE()

batch_size = 32
dropout = 0.2
n_epochs = 50
learning_rate = 1e-3
weight_decay = 1e-6

accuracies = []
for subj, features, labels in fileList:
    
    X_train, X_test, y_train, y_test = train_test_split(features, finger, test_size= 0.5) # 70% training 30% test
        
    test_dataset = EEGDataset(X_test, y_test)
    finger_predictions = []
    for finger in range(5):
        # alter and resample data
        y_train = np.where(labels == finger, 1, 0)
        X_train = X_train.reshape((X_train.shape[0], -1))
        X_res, y_res = sm.fit_resample(X_train, y_train)
        X_res = X_res.reshape((X_res.shape[0], ch, -1))

        # for ConvNet_2D, the input is (batch, channels, height, width)
        input_shape = (X_res.shape[0], 1, X_res.shape[1], X_res.shape[2])
        # for transformer it's.... idk :p
        features = torch.tensor(X_res.reshape(input_shape), dtype=torch.float32).to(device)
        labels = torch.tensor(y_res, dtype=torch.int64).to(device)
        train_dataset = EEGDataset(features, labels)
        train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
        
        model = ConvNet_Finger().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)



        model = train(train_loader, n_epochs)
        pred = predict(train_loader)

        finger_predictions.append(pred)
    finger_predictions = np.array(finger_predictions).T 
    print(finger_predictions.shape)
    X_train, y_train = finger_predictions, labels
    
    model = ConvNet_2D().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

    model = train(train_loader, n_epochs)
    pred = predict(train_loader)
    acc = evaluate(pred, y_train)
    print('acc on test = ', acc)

    test_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
    predict
    print(f'avg accuracy = {np.mean(accuracies)}')


In [7]:
# original 2 class classifier code :D
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
# train neural net
torch.cuda.set_device(3)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# device = 'cpu'
sm = SMOTE()
accuracies = []
for subj, features, labels in fileList:
    finger_predictions = []
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test  
    for finger in range(5):
        y_train_ = np.where(y_train == finger, 1, 0)
        y_test_ = np.where(y_test == finger, 1, 0)
        X_res = X_train.reshape((X_train.shape[0], -1))
        X_res, y_res = sm.fit_resample(X_res, y_train_)
        X_res = X_res.reshape((X_res.shape[0], ch, -1))
        conv_net = ConvNet_Finger().to(device)
        criterion = nn.CrossEntropyLoss()
        conv_net_optimizer = torch.optim.Adam(conv_net.parameters(), lr=0.001)
        # Train the neural network
        for epoch in range(50):
            # print("epoch = ",epoch)
            conv_net.train()
            data = X_res
            targets = y_res
            data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
            data = data.to(device)
            targets = torch.tensor(targets, dtype=torch.int64)
            targets = targets.to(device)

            # Forward pass
            conv_net_predictions = conv_net(data)
            conv_net_loss = criterion(conv_net_predictions, targets)

            # Backward pass
            conv_net_optimizer.zero_grad()
            conv_net_loss.backward()
            conv_net_optimizer.step()

        # Evaluate the neural network
        conv_net.eval()
        correct, total = 0, 0
        with torch.no_grad():
            data = X_test
            targets = y_test_
            data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
            data = data.to(device)
            targets = torch.tensor(targets, dtype=torch.int64)
            targets = targets.to(device)
            conv_net_predictions = conv_net(data)
            _, predicted = torch.max(conv_net_predictions.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            print(binary_confusion(predicted, targets))

        print(f"test accuracy {subj} = ",correct/total)
        accuracies.append(correct/total)

    print(f'avg accuracy = {np.mean(accuracies)}')


KeyboardInterrupt: 

In [11]:
# wip condensed 2 class code that saves the models !

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
# train neural net
torch.cuda.set_device(3)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sm = SMOTE()

batch_size = 32
dropout = 0.2
n_epochs = 50
learning_rate = 1e-3
weight_decay = 1e-6
input_shape = (-1, 1, ch, t)

accuracies = []
for subj, features, labels in fileList:

    #train test split first so WE NEVER EVER LOOK AT TEST BEFORE THE END !!!!
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    X_test = torch.tensor(X_test.reshape(input_shape), dtype=torch.float32).to(device)
    test_dataset = EEGDataset(X_test, y_test)
    
    new_features = []
    # train models for each finger
    for finger in range(5):

        # alter and resample the data
        y_res = np.where(y_train == finger, 1, 0)
        X_res = X_train.reshape((X_train.shape[0], -1))
        X_res, y_res = sm.fit_resample(X_res, y_res)
        X_res = X_res.reshape((X_res.shape[0], ch, -1))
        # X_res = X_train

        # reshape the data for the model type :P.
        # for conv net its (batch, channel, height, width)

        X_res = torch.tensor(X_res.reshape(input_shape), dtype=torch.float32).to(device)
        y_res = torch.tensor(y_res, dtype=torch.int64).to(device)
        res_dataset = EEGDataset(X_res, y_res)
        res_loader = DataLoader(dataset=res_dataset, batch_size=batch_size, shuffle=True)

        # params
        model = ConvNet_Finger().to(device)
        normedWeights = [1 - (x/100) for x in [80, 20]]
        normedWeights = torch.FloatTensor(normedWeights).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        # train and save neural net
        model = train(res_loader, n_epochs)
        torch.save(model.state_dict(), f'pickles/models/{subj}_finger_{finger}.pt')
        # model.load_state_dict(torch.load(f'pickles/models/{subj}_finger_{finger}.pt'))
        # lets peek at the test set. for each class so we gotta modify it...
        two_y = np.where(y_test == finger, 1, 0)
        two_y = torch.tensor(two_y, dtype=torch.int64).to(device)
        pred = predict(torch.tensor(X_test.reshape(input_shape), dtype=torch.float32).to(device))
        acc = evaluate(pred, two_y)
        print(f"test accuracy {subj} {finger} = ", acc)
        print(f'confusion matrix: {binary_confusion(pred, two_y)}')

        # now that model is built, lets use it to classify the og, unbalanced train set:
        # train_dataset = EEGDataset(X_train, y_train)
        # train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        pred = predict(torch.as_tensor(X_train.reshape(input_shape), dtype=torch.float32).to(device))
        new_features.append(pred*finger)

    # use these predictions as input for the final classifer training
    new_features = torch.stack(new_features)
    features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)
    train_dataset = EEGDataset(features, y_train)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    
    model = NeuralNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    # train final classifier
    model.train()
    model = train(train_loader, n_epochs)
    torch.save(model.state_dict(), f'pickles/models/{subj}_final.pt')
    # model.load_state_dict(torch.load(f'pickles/models/{subj}_final.pt'))

    # get features for final classifier
    new_features = []
    for finger in range(5):
        # alter data
        model = ConvNet_Finger().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        y_res = np.where(y_test == finger, 1, 0)
        y_res = torch.tensor(y_res, dtype=torch.int64).to(device)
        model.load_state_dict(torch.load(f'pickles/models/{subj}_finger_{finger}.pt'))
        pred = predict(torch.tensor(X_test.reshape(input_shape), dtype=torch.float32).to(device))
        new_features.append(pred*finger)
    # test final classifier
    y_test = torch.tensor(y_test, dtype=torch.int64).to(device)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
    
    new_features = torch.stack(new_features)
    features = torch.transpose(torch.tensor(new_features), 0, 1).to(device)

    model = NeuralNet().to(device)
    model.load_state_dict(torch.load(f'pickles/models/{subj}_final.pt'))
    print(X_test.shape, y_test.shape)
    acc = evaluate(predict(features), y_test)
    print(f'FINAL accuracy {subj} = {acc} *********************')
    accuracies.append(acc)
print(f'avg accuracy = {np.mean(accuracies)}')

accuracy = 0.7700729927007299
test accuracy subj_A 0 =  0.7700729927007299
confusion matrix: [[0.77007299 0.22992701]
 [0.         0.        ]]
accuracy = 0.7609489051094891
test accuracy subj_A 1 =  0.7609489051094891
confusion matrix: [[0.74817518 0.19525547]
 [0.04379562 0.01277372]]
accuracy = 0.7974452554744526
test accuracy subj_A 2 =  0.7974452554744526
confusion matrix: [[0.79744526 0.20255474]
 [0.         0.        ]]
accuracy = 0.8193430656934306
test accuracy subj_A 3 =  0.8193430656934306
confusion matrix: [[0.81934307 0.18065693]
 [0.         0.        ]]
accuracy = 0.8211678832116789
test accuracy subj_A 4 =  0.8211678832116789
confusion matrix: [[0.82116788 0.17883212]
 [0.         0.        ]]
torch.Size([548, 1, 19, 200]) torch.Size([548])
accuracy = 0.21715328467153286
FINAL accuracy subj_A = 0.21715328467153286 *********************
accuracy = 0.7867132867132867
test accuracy subj_B 0 =  0.7867132867132867
confusion matrix: [[0.71416084 0.15297203]
 [0.06031469 0.07

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [46]:
new_features = torch.tensor(new_features).to(device)
print(new_features.shape)

TypeError: only integer tensors of a single element can be converted to an index

In [9]:
def binary_confusion(y_pred, target):
    # matrix represents [pred 0 targ 0, pred 0 targ 1] [pred 1 targ 0, pred 1 targ 1]
    matrix = [[0, 0],[0, 0]]
    for i in range(y_pred.shape[0]):
        #zero stands for not thumb
        if y_pred[i] == 0 and target[i] == 0:
            matrix[0][0] += 1
        elif y_pred[i] == 0 and target[i] == 1:
            matrix[0][1] += 1
        elif y_pred[i] == 1 and target[i] == 0:
            matrix[1][0] += 1
        elif y_pred[i] == 1 and target[i] == 1:
            matrix[1][1] += 1
    matrix = matrix/np.sum(matrix)
    return matrix

In [ ]:
[[358, 55], [55, 385]]
test accuracy subj_A =  0.8710433763188745
[[767, 68], [140, 806]]
test accuracy subj_B =  0.8832116788321168
[[410, 35], [61, 411]]
test accuracy subj_C =  0.8953107960741549
[[608, 65], [65, 599]]
test accuracy subj_E =  0.9027673896783844
[[608, 41], [90, 613]]
test accuracy subj_F =  0.9031065088757396
[[347, 61], [57, 427]]
test accuracy subj_G =  0.8677130044843049
[[174, 34], [32, 192]]
test accuracy subj_H =  0.8472222222222222
[[398, 77], [63, 362]]
test accuracy subj_I =  0.8444444444444444
avg accuracy = 0.8768524276162802



[[417, 33], [29, 417]]
test accuracy subj_A =  0.9308035714285714
[[817, 99], [102, 844]]
test accuracy subj_B =  0.8920515574650913
[[436, 48], [54, 379]]
test accuracy subj_C =  0.8887677208287895
[[658, 51], [65, 628]]
test accuracy subj_E =  0.9172610556348074
[[660, 52], [54, 647]]
test accuracy subj_F =  0.9249823071479123
[[402, 28], [74, 432]]
test accuracy subj_G =  0.8910256410256411
[[173, 31], [72, 177]]
test accuracy subj_H =  0.7726269315673289
[[371, 75], [90, 408]]
test accuracy subj_I =  0.8252118644067796
avg accuracy = 0.8785968794021977



[[345, 45], [71, 409]]
test accuracy subj_A =  0.8666666666666667
[[810, 166], [95, 752]]
test accuracy subj_B =  0.8568294020844761
[[401, 50], [50, 405]]
test accuracy subj_C =  0.8896247240618101
[[615, 58], [86, 596]]
test accuracy subj_E =  0.8937269372693727
[[586, 82], [87, 613]]
test accuracy subj_F =  0.8764619883040936
[[405, 63], [53, 383]]
test accuracy subj_G =  0.8716814159292036
[[191, 41], [25, 185]]
test accuracy subj_H =  0.8506787330316742
[[396, 42], [57, 418]]
test accuracy subj_I =  0.891566265060241
avg accuracy = 0.8772827584517792



[[371, 22], [80, 407]]
test accuracy subj_A =  0.884090909090909
[[829, 212], [80, 721]]
test accuracy subj_B =  0.8414766558089034
[[380, 34], [46, 452]]
test accuracy subj_C =  0.9122807017543859
[[604, 33], [75, 657]]
test accuracy subj_E =  0.9211102994886778
[[599, 77], [87, 618]]
test accuracy subj_F =  0.8812454742939899
[[389, 53], [55, 419]]
test accuracy subj_G =  0.8820960698689956
[[174, 49], [42, 178]]
test accuracy subj_H =  0.7945823927765236
[[331, 52], [117, 422]]
test accuracy subj_I =  0.8167028199566161
avg accuracy = 0.8746366101838032



[[384, 39], [48, 414]]
test accuracy subj_A =  0.9016949152542373
[[855, 104], [102, 780]]
test accuracy subj_B =  0.8881042911461162
[[443, 34], [30, 401]]
test accuracy subj_C =  0.9295154185022027
[[634, 44], [58, 638]]
test accuracy subj_E =  0.925764192139738
[[590, 66], [78, 651]]
test accuracy subj_F =  0.896028880866426
[[385, 92], [81, 359]]
test accuracy subj_G =  0.8113413304252999
[[210, 25], [31, 178]]
test accuracy subj_H =  0.8738738738738738
[[393, 48], [67, 416]]
test accuracy subj_I =  0.8755411255411255
avg accuracy = 0.877255888840768